In [ ]:
import pandas as pd
import numpy as np

import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df=pd.read_csv('/content/spam_ham_dataset.csv')

In [ ]:
df.head(1)

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0


In [ ]:
df['text']=df['text'].apply(lambda x:x.replace('\r\n',' '))

In [ ]:
df['text'].iloc[2]

"Subject: neon retreat ho ho ho , we ' re around to that most wonderful time of the year - - - neon leaders retreat time ! i know that this time of year is extremely hectic , and that it ' s tough to think about anything past the holidays , but life does go on past the week of december 25 through january 1 , and that ' s what i ' d like you to think about for a minute . on the calender that i handed out at the beginning of the fall semester , the retreat was scheduled for the weekend of january 5 - 6 . but because of a youth ministers conference that brad and dustin are connected with that week , we ' re going to change the date to the following weekend , january 12 - 13 . now comes the part you need to think about . i think we all agree that it ' s important for us to get together and have some time to recharge our batteries before we get to far into the spring semester , but it can be a lot of trouble and difficult for us to get away without kids , etc . so , brad came up with a pote

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5171 non-null   int64 
 1   label       5171 non-null   object
 2   text        5171 non-null   object
 3   label_num   5171 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.7+ KB


In [ ]:
df.label_num.value_counts()

label_num
0    3672
1    1499
Name: count, dtype: int64

In [ ]:
stemmer = PorterStemmer()
corpus=[]

stopwords_set = set(stopwords.words('english'))

for i in range(len(df)):
  text=df['text'].iloc[i]
  text=text.translate(str.maketrans('','',string.punctuation)).split()
  text=[stemmer.stem(word) for word in text if word not in stopwords_set]
  text=''.join(text)
  corpus.append(text)

In [ ]:
df.text.iloc[2]

"Subject: neon retreat ho ho ho , we ' re around to that most wonderful time of the year - - - neon leaders retreat time ! i know that this time of year is extremely hectic , and that it ' s tough to think about anything past the holidays , but life does go on past the week of december 25 through january 1 , and that ' s what i ' d like you to think about for a minute . on the calender that i handed out at the beginning of the fall semester , the retreat was scheduled for the weekend of january 5 - 6 . but because of a youth ministers conference that brad and dustin are connected with that week , we ' re going to change the date to the following weekend , january 12 - 13 . now comes the part you need to think about . i think we all agree that it ' s important for us to get together and have some time to recharge our batteries before we get to far into the spring semester , but it can be a lot of trouble and difficult for us to get away without kids , etc . so , brad came up with a pote

In [ ]:
vectorizer = CountVectorizer()

x=vectorizer.fit_transform(corpus).toarray()
y=df.label_num


In [ ]:
x[0]

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


# Define classifiers
classifiers = {
    "Naive Bayes": GaussianNB(),
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier()
}

# Dictionary to store cross-validation scores
cv_scores = {}

# Apply cross-validation for each classifier
for clf_name, clf in classifiers.items():
    scores = cross_val_score(clf, x, y, cv=5)  # 5-fold cross-validation
    cv_scores[clf_name] = scores.mean()

# Create a DataFrame to display cross-validation scores
cv_scores_df = pd.DataFrame(list(cv_scores.items()), columns=['Classifier', 'Mean Cross-Val Score'])

print(cv_scores_df)


      Classifier  Mean Cross-Val Score
0    Naive Bayes              0.363374
1            SVM              0.721137
2  Decision Tree              0.721137
3  Random Forest              0.721137
